In [1]:
import numpy as np
import string
import nltk 
import os
import re
import emoji
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import f1_score

In [2]:
df_train1_x=pd.read_csv(r"C:\Users\NehaS\Desktop\HW\CS 594 Project\Data\training_set_1_tweets_csv.csv", sep="\t" )
df_test1_x=pd.read_csv(r"C:\Users\NehaS\Desktop\HW\CS 594 Project\Data\evaluation_set_tweets_csv.csv", sep="\t" )
df_trainY=pd.read_csv(r"C:\Users\NehaS\Desktop\HW\CS 594 Project\Data\training_set_1_ids.txt", sep="\t", header=None )
df_testY=pd.read_csv(r"C:\Users\NehaS\Desktop\HW\CS 594 Project\Data\evaluation_set_ids.txt", sep="\t", header=None)
df_train1=df_train1_x.merge(df_trainY, how = "inner", left_on='id', right_on=0)
df_test1=df_test1_x.merge(df_testY, how = "inner", left_on='id', right_on=0)
df_test1=df_test1.dropna(subset=[2])
df_train1=df_train1.dropna(subset=[2])

In [3]:
nltk.download('words')
words = set(nltk.corpus.words.words())
stemmer=PorterStemmer()
def preprocess(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet)
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) 
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) 
    tweet = tweet.replace("#", "").replace("_", " ")
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if w.lower() in words or not w.isalpha())
    stems = [stemmer.stem(item) for item in tweet]
    return tweet
df_train1['text'] = df_train1['text'].map(lambda x: preprocess(x))
df_test1['text']=df_test1['text'].map(lambda x:preprocess(x))

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\NehaS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit_transform(df_train1['text'])
X_train1=vectorizer.transform(df_train1['text'])
X_test1=vectorizer.transform(df_test1['text'])

In [5]:
lr_model = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=1000)
model=lr_model.fit(X_train1,df_train1[2])
Y_pred=model.predict(X_test1)
df_test1[2]=df_test1[2].astype(np.int64)
f1_score(df_test1[2],Y_pred, average='micro')

[LibLinear]

0.5868123992116108